<div style="text-align: right">Miguel Angel Sosvilla Luis, 4 de enero de 2016</div> 
# Cálculo de Probabilidades en el Reparto de Cartas

A partir del notebook de Peter Norvig [Probability, Paradox, and the Reasonable Person Principle](http://nbviewer.ipython.org/url/norvig.com/ipython/Probability.ipynb) un par de ejemplos con el reparto de cartas de una baraja.

In [1]:
from fractions import Fraction
from __future__ import division
def P(event, space): 
    """The probability of an event, given a sample space of equiprobable outcomes.
    event can be either a set of outcomes, or a predicate (true for outcomes in the event)."""
    if callable(event):
        event = such_that(event, space)
    return Fraction(len(event & space), len(space))

def such_that(predicate, collection): 
    "The subset of elements in the collection for which the predicate is true."
    return {e for e in collection if predicate(e)}

In [2]:
from IPython.display import HTML

def table(event,space,n=3):
    """Display sample space in a table, color-coded: green if event is true; yellow otherwise."""
    # n is the number of characters for one card
    firsts = sorted(set(outcome[:n] for outcome in space))
    return HTML('<table>' +
                cat(row(first, space, event) for first in firsts) +
                '</table>' + 
                str(P(event, space)))

def row(first, space, event):
    "Display a row where an older child is paired with each of the possible younger children."
    thisrow = sorted(outcome for outcome in space if outcome.startswith(first))
    return '<tr>' + cat(cell(outcome, event) for outcome in thisrow) + '</tr>'

def cell(outcome, event): 
    "Display outcome in appropriate color."
    color = ('lightgreen' if event(outcome) else 'yellow')
    return '<td style="background-color: {}">{}</td>'.format(color, outcome)    

cat = ''.join

In [3]:
import itertools
import re

palos = {"O","C","E","B"}

def create_baraja_cartas(palos,np=10):
    np = 10 # Número de cartas en un palo
    baraja_cartas = sorted({palo+str(numero).zfill(2)
                  for palo in palos
                  for numero in range (1,np+1)})
    return baraja_cartas

def manos(baraja_cartas,n):
    # n es el numero de cartas de la mano
    
    return {''.join(m) for m in itertools.combinations(baraja_cartas,n)}

## Problema 1: Probabilidad de sacar toda la mano del mismo palo
La condición para las manos de un mismo palo es:

In [4]:
def mismo_palo(outcome):
    palos = set (re.sub('\d', '', outcome))
    return len(palos) == 1

Ahora creamos el baraja de cartas, todas las manos de dos cartas y calculamos la probabilidad:

In [5]:
baraja_cartas=create_baraja_cartas(palos,10)
todas_manos = manos(baraja_cartas,2)

print (P(mismo_palo, todas_manos))

3/13


In [6]:
table(mismo_palo,todas_manos)

## Problema 2: Probabilidad de sacar todo ases

In [7]:
def todo_ases(outcome):
    numeros = set (re.sub('['+ ''.join(palos) +']', '', outcome))
    
    numeros = re.sub('['+ ''.join(palos) +']', '', outcome)
    numeros_set =set()
    for n in range(0,len(numeros),2):
        numeros_set.add (numeros[n:n+2])

    return numeros_set == {'01'}


In [8]:
table(todo_ases,todas_manos)

In [9]:
print (P(todo_ases, todas_manos))

1/130


## Problema 3: Probabilidad de sacar dos ases usando la Probabilidad Compuesta
Calculo ahora la probabilidad de sacar dos ases, pero como dos eventos dependientes. Primero saco un as y despues otro:

In [10]:
#Probabilidad de sacar un as
baraja_cartas=create_baraja_cartas(palos,10)
todas_manos = manos(baraja_cartas,1)
probabilidad_primer_as = P(todo_ases, todas_manos)
print ("Probabilidad de sacar un primer as:", probabilidad_primer_as)

#Quito el as de la baraja y veo la probabilidad de sacar otro
baraja_cartas.remove('B01')
todas_manos = manos(baraja_cartas,1)
probabilidad_segundo_as = P(todo_ases, todas_manos)
print ("Probabilidad de sacar el segundo as:", probabilidad_segundo_as)

Probabilidad de sacar un primer as: 1/10
Probabilidad de sacar el segundo as: 1/13


Para calcular la **probabilidad compuesta** multiplico la probabilidad de los sucesos dependientes (sacar el primer as y a continuación sacar el segundo):

In [11]:
probabilidad_compuesta = probabilidad_primer_as * probabilidad_segundo_as
print ("Probabilidad Compuesta: %s x %s = %s" % (probabilidad_primer_as,probabilidad_segundo_as,probabilidad_compuesta))

Probabilidad Compuesta: 1/10 x 1/13 = 1/130


Es igual a la que calculamos sobre todas las manos de dos cartas formadas por dos ases.